In [7]:
from transformers import Wav2Vec2FeatureExtractor, WavLMForXVector
from datasets import load_dataset
import torch

# Cargamos el dataset CIEMPIESS
dataset = load_dataset("ciempiess/ciempiess_test")

0000.parquet:   0%|          | 0.00/512M [00:00<?, ?B/s]

c:\Users\mrcom\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\mrcom\.cache\huggingface\hub\datasets--ciempiess--ciempiess_test. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


0001.parquet:   0%|          | 0.00/56.7M [00:00<?, ?B/s]

Generating test split:   0%|          | 0/3558 [00:00<?, ? examples/s]

In [2]:
# Cargamos el modelo preentrenado de Hugging Face y el extractor de caracteristicas
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained('microsoft/wavlm-base-sv')
model = WavLMForXVector.from_pretrained('microsoft/wavlm-base-sv')


preprocessor_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

c:\Users\mrcom\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\mrcom\.cache\huggingface\hub\models--microsoft--wavlm-base-sv. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/58.6k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/405M [00:00<?, ?B/s]

In [10]:
print(dataset)

DatasetDict({
    test: Dataset({
        features: ['audio_id', 'audio', 'speaker_id', 'gender', 'duration', 'normalized_text'],
        num_rows: 3558
    })
})


In [11]:
# Accedemos al subconjunto 'test' del dataset
test_dataset = dataset['test']

In [ ]:
"""
reference_speaker_id = 'M_07'
reference_audio = test_dataset.filter(lambda x: x["speaker_id"] == reference_speaker_id)["audio"][0]["array"]

# Extraer las características del audio de referencia
reference_input = feature_extractor(reference_audio, return_tensors="pt")
reference_embedding = model(**reference_input).embeddings
reference_embedding = torch.nn.functional.normalize(reference_embedding, dim=-1).cpu()

# Función para comparar la voz del hablante con los otros hablantes
def compare_speakers(audio1, audio2, threshold=0.86 ):
    cosine_sim = torch.nn.CosineSimilarity(dim=-1)
    similarity = cosine_sim(audio1, audio2)
    return similarity >= threshold

# Comparar la voz del hablante M_07 con todos los demás hablantes
for sample in test_dataset:
    if sample["speaker_id"] != reference_speaker_id:
        other_audio = sample["audio"]["array"]
        other_input = feature_extractor(other_audio, return_tensors="pt")
        other_embedding = model(**other_input).embeddings
        other_embedding = torch.nn.functional.normalize(other_embedding, dim=-1).cpu()

        if compare_speakers(reference_embedding, other_embedding):
            print(f"La voz del hablante con ID {reference_speaker_id} es similar a la del hablante {sample['speaker_id']}.")
        else:
            print(f"La voz del hablante con ID {reference_speaker_id} NO coincide con la del hablante {sample['speaker_id']}.")
"""

In [19]:
import librosa

# Sample rate deseado (16000 Hz)
sampling_rate = 16000

# Cargamos y filtramos el audio de referencia con el sampling_rate
reference_speaker_id = 'M_07'
reference_audio = test_dataset.filter(lambda x: x["speaker_id"] == reference_speaker_id)["audio"][0]["array"]

# Extraer las características del audio de referencia
reference_input = feature_extractor(reference_audio, sampling_rate=sampling_rate, return_tensors="pt")
reference_embedding = model(**reference_input).embeddings
reference_embedding = torch.nn.functional.normalize(reference_embedding, dim=-1).cpu()

# Funcion para comparar la voz del hablante con los otros hablantes
def compare_speakers(audio1, audio2, threshold=0.86):
    cosine_sim = torch.nn.CosineSimilarity(dim=-1)
    similarity = cosine_sim(audio1, audio2)
    return similarity >= threshold

# Lista de los hablantes ya comparados
compared_speakers = []

# Comparamos la voz del hablante M_07 con todos los demas hablantes
for sample in test_dataset:
    if sample["speaker_id"] != reference_speaker_id and sample["speaker_id"] not in compared_speakers:
        other_audio = sample["audio"]["array"]
        
        # Extraemos las caracteristicas del otro hablante asegurando el sampling_rate
        other_input = feature_extractor(other_audio, sampling_rate=sampling_rate, return_tensors="pt")
        other_embedding = model(**other_input).embeddings
        other_embedding = torch.nn.functional.normalize(other_embedding, dim=-1).cpu()

        # Comparamos las voces
        if compare_speakers(reference_embedding, other_embedding):
            print(f"La voz del hablante con ID {reference_speaker_id} es similar a la del hablante {sample['speaker_id']}.")
        else:
            print(f"La voz del hablante con ID {reference_speaker_id} NO coincide con la del hablante {sample['speaker_id']}.")
        
        compared_speakers.append(sample["speaker_id"])

La voz del hablante con ID M_07 NO coincide con la del hablante F_01.
La voz del hablante con ID M_07 NO coincide con la del hablante F_05.
La voz del hablante con ID M_07 NO coincide con la del hablante F_07.
La voz del hablante con ID M_07 NO coincide con la del hablante F_04.
La voz del hablante con ID M_07 es similar a la del hablante M_01.
La voz del hablante con ID M_07 NO coincide con la del hablante F_08.
La voz del hablante con ID M_07 es similar a la del hablante M_10.
La voz del hablante con ID M_07 NO coincide con la del hablante M_09.
La voz del hablante con ID M_07 NO coincide con la del hablante M_03.
La voz del hablante con ID M_07 NO coincide con la del hablante M_04.
La voz del hablante con ID M_07 NO coincide con la del hablante F_03.
La voz del hablante con ID M_07 NO coincide con la del hablante F_10.
La voz del hablante con ID M_07 NO coincide con la del hablante F_06.
La voz del hablante con ID M_07 NO coincide con la del hablante F_09.
La voz del hablante con ID